## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
#Ignore future version warning (version warning)

In [2]:
#import modules
import sklearn as sk
import seaborn as sns
import numpy as np
import keras_tuner as kt
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# REGRESSION

## Data Preprocessing & Feature selection

In [3]:
def get_best_n(df_input,model):
    scaler = StandardScaler()
    best = 0
    n_pca = 0
    scaler.fit(df_input)
    scaled_data = scaler.transform(df_input)
    for i, val in enumerate(range(len(scaled_data[1]))):
        if i == 0:
            i = 1
        pca = PCA(n_components = i)
        pca.fit(scaled_data)
        x_pca = pca.transform(scaled_data)
        nX_train, nX_test, ny_train, ny_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=123)
        n_model = model
        n_model = n_model.fit(nX_train, ny_train)
        val = n_model.score(nX_test,ny_test)
        if abs(val) > abs(best):
            best = val
            n_pca = i
    return (n_pca,best)

In [4]:
def best_pre_process(data,model,to_drop):
    proc_1 = get_best_n(data,model)
    val_1 = proc_1[1]
    df_proc_2 = data.drop(to_drop,axis=1)
    proc_2 = get_best_n(df_proc_2,model)
    val_2 = proc_2[1]
    if val_1 > val_2:
        return (proc_1[0],"Only PCA")
    else:
        return (proc_2[0],"Pairwise & PCA")

In [5]:
df_regressionData = pd.read_excel("trainDataset.xls", usecols = ['RelapseFreeSurvival (outcome)','Age','original_shape_Elongation','original_shape_Flatness','original_shape_LeastAxisLength','original_shape_MajorAxisLength','original_shape_Maximum2DDiameterColumn','original_shape_Maximum2DDiameterRow','original_shape_Maximum2DDiameterSlice','original_shape_Maximum3DDiameter','original_shape_MeshVolume','original_shape_MinorAxisLength','original_shape_Sphericity','original_shape_SurfaceArea','original_shape_SurfaceVolumeRatio','original_shape_VoxelVolume','original_firstorder_10Percentile','original_firstorder_90Percentile','original_firstorder_Energy','original_firstorder_Entropy','original_firstorder_InterquartileRange','original_firstorder_Kurtosis','original_firstorder_Maximum','original_firstorder_MeanAbsoluteDeviation','original_firstorder_Mean','original_firstorder_Median','original_firstorder_Minimum','original_firstorder_Range','original_firstorder_RobustMeanAbsoluteDeviation','original_firstorder_RootMeanSquared','original_firstorder_Skewness','original_firstorder_TotalEnergy','original_firstorder_Uniformity','original_firstorder_Variance','original_glcm_Autocorrelation','original_glcm_ClusterProminence','original_glcm_ClusterShade','original_glcm_ClusterTendency','original_glcm_Contrast','original_glcm_Correlation','original_glcm_DifferenceAverage','original_glcm_DifferenceEntropy','original_glcm_DifferenceVariance','original_glcm_Id','original_glcm_Idm','original_glcm_Idmn','original_glcm_Idn','original_glcm_Imc1','original_glcm_Imc2','original_glcm_InverseVariance','original_glcm_JointAverage','original_glcm_JointEnergy','original_glcm_JointEntropy','original_glcm_MCC','original_glcm_MaximumProbability','original_glcm_SumAverage','original_glcm_SumEntropy','original_glcm_SumSquares','original_gldm_DependenceEntropy','original_gldm_DependenceNonUniformity','original_gldm_DependenceNonUniformityNormalized','original_gldm_DependenceVariance','original_gldm_GrayLevelNonUniformity','original_gldm_GrayLevelVariance','original_gldm_HighGrayLevelEmphasis','original_gldm_LargeDependenceEmphasis','original_gldm_LargeDependenceHighGrayLevelEmphasis','original_gldm_LargeDependenceLowGrayLevelEmphasis','original_gldm_LowGrayLevelEmphasis','original_gldm_SmallDependenceEmphasis','original_gldm_SmallDependenceHighGrayLevelEmphasis','original_gldm_SmallDependenceLowGrayLevelEmphasis','original_glrlm_GrayLevelNonUniformity','original_glrlm_GrayLevelNonUniformityNormalized','original_glrlm_GrayLevelVariance','original_glrlm_HighGrayLevelRunEmphasis','original_glrlm_LongRunEmphasis','original_glrlm_LongRunHighGrayLevelEmphasis','original_glrlm_LongRunLowGrayLevelEmphasis','original_glrlm_LowGrayLevelRunEmphasis','original_glrlm_RunEntropy','original_glrlm_RunLengthNonUniformity','original_glrlm_RunLengthNonUniformityNormalized','original_glrlm_RunPercentage','original_glrlm_RunVariance','original_glrlm_ShortRunEmphasis','original_glrlm_ShortRunHighGrayLevelEmphasis','original_glrlm_ShortRunLowGrayLevelEmphasis','original_glszm_GrayLevelNonUniformity','original_glszm_GrayLevelNonUniformityNormalized','original_glszm_GrayLevelVariance','original_glszm_HighGrayLevelZoneEmphasis','original_glszm_LargeAreaEmphasis','original_glszm_LargeAreaHighGrayLevelEmphasis','original_glszm_LargeAreaLowGrayLevelEmphasis','original_glszm_LowGrayLevelZoneEmphasis','original_glszm_SizeZoneNonUniformity','original_glszm_SizeZoneNonUniformityNormalized','original_glszm_SmallAreaEmphasis','original_glszm_SmallAreaHighGrayLevelEmphasis','original_glszm_SmallAreaLowGrayLevelEmphasis','original_glszm_ZoneEntropy','original_glszm_ZonePercentage','original_glszm_ZoneVariance','original_ngtdm_Busyness','original_ngtdm_Coarseness','original_ngtdm_Complexity','original_ngtdm_Contrast','original_ngtdm_Strength'])
df_regressionData = df_regressionData.replace(999, np.nan)

In [6]:
df_regressionData.dtypes.unique()

array([dtype('float64'), dtype('int64')], dtype=object)

In [7]:
df_regressionData.shape

(400, 109)

In [8]:
df_regressionData.isna().sum().sum()

0

In [9]:
scaler = MinMaxScaler()
df_regressionData = pd.DataFrame(scaler.fit_transform(df_regressionData),columns=df_regressionData.columns,index=df_regressionData.index)

In [10]:
df_result = df_regressionData.iloc[:, 0]
df_input = df_regressionData.iloc[:, 1:]

In [11]:
to_drop = ['original_shape_Maximum2DDiameterSlice',
            'original_shape_SurfaceArea',
            'original_firstorder_Energy',
            'original_firstorder_Mean',
            'original_firstorder_MeanAbsoluteDeviation',
            'original_firstorder_Uniformity',
            'original_firstorder_Entropy',
            'original_firstorder_Median', 
            'original_shape_VoxelVolume',
            'original_glcm_ClusterProminence',
            'original_glcm_ClusterTendency',
            'original_firstorder_Variance',
            'original_glcm_Contrast',
            'original_glcm_DifferenceAverage',
            'original_glcm_DifferenceVariance',
            'original_glcm_Id',
            'original_glcm_Idm',
            'original_glcm_Idmn',
            'original_glcm_DifferenceEntropy',
            'original_glcm_Idn',
            'original_glcm_JointEnergy',
            'original_glcm_JointEntropy',
            'original_glcm_SumEntropy',
            'original_glcm_SumSquares',
            'original_glcm_MaximumProbability',
            'original_gldm_LowGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformityNormalized',
            'original_gldm_HighGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformity',
            'original_glrlm_LongRunHighGrayLevelEmphasis',
            'original_gldm_GrayLevelNonUniformity',
            'original_gldm_GrayLevelVariance',
            'original_gldm_LargeDependenceEmphasis',
            'original_gldm_LargeDependenceHighGrayLevelEmphasis',
            'original_gldm_LargeDependenceLowGrayLevelEmphasis',
            'original_glrlm_GrayLevelVariance',
            'original_glszm_LowGrayLevelZoneEmphasis',
            'original_glrlm_ShortRunEmphasis',
            'original_glszm_SmallAreaLowGrayLevelEmphasis',
            'original_glrlm_LongRunEmphasis',
            'original_glszm_LargeAreaHighGrayLevelEmphasis',
            'original_ngtdm_Complexity',
            'original_ngtdm_Coarseness',
            'original_ngtdm_Contrast',
            'original_glcm_SumAverage'
            ]

In [12]:
df_corr_drop = df_input.drop(to_drop,axis=1)

## Linear Regression

In [13]:
result = best_pre_process(df_input,LinearRegression(),to_drop)
result

(38, 'Pairwise & PCA')

In [14]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [15]:
xR_train, xR_test, yR_train, yR_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=1)

In [16]:
gs_linreg = GridSearchCV(LinearRegression(),{
    'fit_intercept': [False,True], #Default = False
    'n_jobs': [0,1,10,50],
}, cv=10, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_linreg.fit(xR_train, yR_train)

GridSearchCV(cv=10, estimator=LinearRegression(),
             param_grid={'fit_intercept': [False, True],
                         'n_jobs': [0, 1, 10, 50]},
             scoring='neg_mean_squared_error')

In [17]:
dfgs = pd.DataFrame(gs_linreg.cv_results_)
dfgs[['param_fit_intercept','param_n_jobs', 'mean_test_score']]

,param_fit_intercept,param_n_jobs,mean_test_score
0,False,0,-2.10999
1,False,1,-2.10999
2,False,10,-2.10999
3,False,50,-2.10999
4,True,0,-0.05807
5,True,1,-0.05807
6,True,10,-0.05807
7,True,50,-0.05807


## Support Vector Machines

In [18]:
result = best_pre_process(df_input,SVR(),to_drop)
result

(15, 'Pairwise & PCA')

In [19]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [20]:
xR_train, xR_test, yR_train, yR_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=1)

In [21]:
gs_SVM = GridSearchCV(SVR(),{
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree' : [2,3], 
}, cv=10, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_SVM.fit(xR_train,yR_train)

GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'degree': [2, 3],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='neg_mean_squared_error')

In [22]:
dfgs = pd.DataFrame(gs_SVM.cv_results_)
dfgs[['param_kernel','param_degree', 'mean_test_score']]

,param_kernel,param_degree,mean_test_score
0,linear,2,-0.040556
1,poly,2,-0.059169
2,rbf,2,-0.039847
3,sigmoid,2,-2.543947
4,linear,3,-0.040556
5,poly,3,-0.060234
6,rbf,3,-0.039847
7,sigmoid,3,-2.543947


In [23]:
#Eval on hyper-parameters of LinearRegression
gs_LinearSVM = GridSearchCV(LinearSVR(),{
    'tol' : [0.00001, 0.0001, 0.001], 
    'loss' : ['epsilon_insensitive', 'squared_epsilon_insensitive'] 
}, cv=10, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_LinearSVM.fit(xR_train,yR_train)

GridSearchCV(cv=10, estimator=LinearSVR(),
             param_grid={'loss': ['epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'tol': [1e-05, 0.0001, 0.001]},
             scoring='neg_mean_squared_error')

In [24]:
dfgs = pd.DataFrame(gs_LinearSVM.cv_results_)
dfgs[['param_loss', 'param_tol', 'mean_test_score']]
#Having 'kernel : poly' with a degree of 2 improves the accuracy 

,param_loss,param_tol,mean_test_score
0,epsilon_insensitive,0.00001,-0.040779
1,epsilon_insensitive,0.0001,-0.040805
2,epsilon_insensitive,0.001,-0.041001
3,squared_epsilon_insensitive,0.00001,-0.039380
4,squared_epsilon_insensitive,0.0001,-0.039380
5,squared_epsilon_insensitive,0.001,-0.039391


## Decision Trees

In [25]:
result = best_pre_process(df_input,DecisionTreeRegressor(),to_drop)
result

(14, 'Pairwise & PCA')

In [26]:
#Peforming this PCA yields worse results (do not use)

# pca = PCA(n_components = result[0])
# if result[1] == "Pairwise & PCA":
#     pca.fit(df_corr_drop)
#     x_pca = pca.transform(df_corr_drop)
# else:
#     pca.fit(df_input)
#     x_pca = pca.transform(df_input)

In [27]:
xR_train, xR_test, yR_train, yR_test = train_test_split(df_input,df_result, test_size=0.3, random_state=1)

In [28]:
gs_DTR = GridSearchCV(DecisionTreeRegressor(),{
    'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'],
    'splitter' : ['best','random'], 
}, cv=10, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_DTR.fit(xR_train,yR_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [29]:
dfgs = pd.DataFrame(gs_DTR.cv_results_)
dfgs[['param_criterion', 'param_splitter', 'mean_test_score']]

,param_criterion,param_splitter,mean_test_score
0,squared_error,best,-0.068285
1,squared_error,random,-0.072046
2,friedman_mse,best,-0.072744
3,friedman_mse,random,-0.081774
4,absolute_error,best,-0.072803
5,absolute_error,random,-0.083580
6,poisson,best,-0.058856
7,poisson,random,-0.075235


##  Multi-Layer Perceptron Neural Network.

In [30]:
result = best_pre_process(df_input,MLPRegressor(),to_drop)
result

(41, 'Pairwise & PCA')

In [31]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [32]:
xR_train, xR_test, yR_train, yR_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=1)

In [33]:
#Eval on hyper-parameters of MLPRegressor
gs_MLP = GridSearchCV(MLPRegressor(),{
    'activation': ['identity', 'logistic', 'tanh', 'relu'], #default =relu
    'solver': ['lbfgs','sgd','adam'] #default =adam
}, cv=5, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_MLP.fit(xR_train,yR_train)

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             scoring='neg_mean_squared_error')

In [34]:
dfgs = pd.DataFrame(gs_MLP.cv_results_)
dfgs[['param_activation', 'param_solver', 'mean_test_score']]

,param_activation,param_solver,mean_test_score
0,identity,lbfgs,-0.092817
1,identity,sgd,-0.047495
2,identity,adam,-0.042203
3,logistic,lbfgs,-0.047702
4,logistic,sgd,-0.038991
5,logistic,adam,-0.038712
6,tanh,lbfgs,-0.127049
7,tanh,sgd,-0.045941
8,tanh,adam,-0.042966
9,relu,lbfgs,-0.105829


In [35]:
gs_MLP2 = GridSearchCV( MLPRegressor(),{
    'solver': ['adam'],
    'max_fun':[15000,20000,10000,25000] #default is 15000 and only applys to lbfgs
}, cv=5, return_train_score=False,scoring = 'neg_mean_squared_error' )
gs_MLP2.fit(xR_train,yR_train)

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'max_fun': [15000, 20000, 10000, 25000],
                         'solver': ['adam']},
             scoring='neg_mean_squared_error')

In [36]:
dfgs = pd.DataFrame(gs_MLP2.cv_results_)
dfgs[['param_solver', 'param_max_fun', 'mean_test_score']]

,param_solver,param_max_fun,mean_test_score
0,adam,15000,-0.044715
1,adam,20000,-0.043022
2,adam,10000,-0.043091
3,adam,25000,-0.042868


# CLASSIFICATION

## Data Handling

In [37]:
df_raw_data = pd.read_excel('trainDataset.xls')
df_raw_data = df_raw_data.drop(['RelapseFreeSurvival (outcome)', 'ID'], axis = 1)

df_raw = df_raw_data.replace(999, np.nan)
df_raw['TrippleNegative'] = df_raw['TrippleNegative'].fillna(df_raw['TrippleNegative'].mode()[0])
df_raw['ChemoGrade'] = df_raw['ChemoGrade'].fillna(df_raw['ChemoGrade'].mode()[0])
df_raw['Proliferation'] = df_raw['Proliferation'].fillna(df_raw['Proliferation'].mode()[0])
df_raw['HistologyType'] = df_raw['HistologyType'].fillna(df_raw['HistologyType'].mode()[0])
df_raw['LNStatus'] = df_raw['LNStatus'].fillna(df_raw['LNStatus'].mode()[0])
df_raw['TumourStage'] = df_raw['TumourStage'].fillna(df_raw['TumourStage'].mode()[0])
df_raw = df_raw.dropna()

target = df_raw['pCR (outcome)']
learning = df_raw.drop('pCR (outcome)', axis = 1)
cols = list(learning.columns)
scaler = StandardScaler()
scaler.fit(learning)
scaled = scaler.transform(learning)
data = pd.DataFrame(scaled, columns=cols)

pca = PCA(.95)
pca.fit(data)
reduced = pca.transform(data)

dfClass = pd.DataFrame(reduced)

In [38]:
xC_train = dfClass.to_numpy()
yC_train = target

## Logistic Regression

In [39]:
gs_logreg = GridSearchCV(LogisticRegression(solver = 'liblinear'),{
    'penalty': ['l1','l2'] #default = l2
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_logreg.fit(xC_train, yC_train)

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'penalty': ['l1', 'l2']}, scoring='accuracy')

In [40]:
dfgs = pd.DataFrame(gs_logreg.cv_results_)
dfgs[['param_penalty', 'mean_test_score']]

,param_penalty,mean_test_score
0,l1,0.753846
1,l2,0.738624


## Support Vector Machines

In [41]:
gs_SVC = GridSearchCV(SVC(),{
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #default rbf
    'degree' : [2,3], #default = 3, only applys to kernel: poly
}, cv=10, return_train_score=False, scoring = 'accuracy')
gs_SVC.fit(xC_train,yC_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'degree': [2, 3],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [42]:
dfgs = pd.DataFrame(gs_SVC.cv_results_)
dfgs[['param_kernel','param_degree', 'mean_test_score']]
#Having 'kernel : poly' with a degree of 2 improves the accuracy 

,param_kernel,param_degree,mean_test_score
0,linear,2,0.756410
1,poly,2,0.751282
2,rbf,2,0.748718
3,sigmoid,2,0.743718
4,linear,3,0.756410
5,poly,3,0.758782
6,rbf,3,0.748718
7,sigmoid,3,0.743718


In [43]:
#Eval on hyper-parameters of LinearRegression
gs_LinearSVC = GridSearchCV(LinearSVC(),{
    'tol' : [0.00001, 0.0001, 0.001], 
    'loss' : ['epsilon_insensitive', 'squared_epsilon_insensitive'] 
}, cv=10, return_train_score=False, scoring = 'accuracy')
gs_LinearSVC.fit(xC_train,yC_train)

GridSearchCV(cv=10, estimator=LinearSVC(),
             param_grid={'loss': ['epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'tol': [1e-05, 0.0001, 0.001]},
             scoring='accuracy')

In [44]:
dfgs = pd.DataFrame(gs_LinearSVC.cv_results_)
dfgs[['param_loss', 'param_tol', 'mean_test_score']]
#Having 'kernel : poly' with a degree of 2 improves the accuracy 

,param_loss,param_tol,mean_test_score
0,epsilon_insensitive,0.00001,0.241026
1,epsilon_insensitive,0.0001,0.241026
2,epsilon_insensitive,0.001,0.241026
3,squared_epsilon_insensitive,0.00001,0.256154
4,squared_epsilon_insensitive,0.0001,0.256154
5,squared_epsilon_insensitive,0.001,0.263782


## Decision Trees

In [45]:
gs_RFC = GridSearchCV(RandomForestClassifier(),{
    'criterion' : ['gini','entropy'],
    'max_features' : ['sqrt','log2']
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_RFC.fit(xC_train,yC_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2']},
             scoring='accuracy')

In [46]:
dfgs = pd.DataFrame(gs_RFC.cv_results_)
dfgs[['param_criterion','param_max_features','mean_test_score']]

,param_criterion,param_max_features,mean_test_score
0,gini,sqrt,0.751152
1,gini,log2,0.756183
2,entropy,sqrt,0.746089
3,entropy,log2,0.748685


## Multi-Layer Perceptron Neural Network

In [47]:
mlpc = MLPClassifier(max_iter=2500, alpha=0.00001, hidden_layer_sizes=12, random_state=3)
mlpc.fit(xC_train, yC_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=12, max_iter=2500, random_state=3)

In [49]:
gs_MLPC = GridSearchCV(MLPClassifier(),{
    'activation': ['identity','logistic','tanh','relu']
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_MLPC.fit(xC_train,yC_train)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic', 'tanh',
                                        'relu']},
             scoring='accuracy')

In [50]:
dfgs = pd.DataFrame(gs_MLPC.cv_results_)
dfgs[['param_activation','mean_test_score']]

,param_activation,mean_test_score
0,identity,0.730932
1,logistic,0.728400
2,tanh,0.764005
3,relu,0.743557


In [51]:
features = list(X)
pca = PCA(n_components=10)
Xs_pca = pca.fit_transform(X)
Xs_pca=Xs_pca[:,0:5] #retain the first 5 PC
y.head()

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(5, input_dim=5,activation="relu"))    
    
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=10, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    model.summary()
    return model
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)



tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')




tuner.search(X_train, y_train, epochs=5, validation_split=0.2, callbacks=[stop_early], verbose=2)



best_hps=tuner.get_best_hyperparameters()[0]
h_model = tuner.hypermodel.build(best_hps)
h_model.fit(X_train, y_train, epochs=25, validation_split=0.2, callbacks=[stop_early], verbose=2)



h_model.evaluate(X_test, y_test, return_dict=True)

NameError: name 'X' is not defined